In [1]:
using LinearAlgebra

In [2]:
# From terms_and_polynomials.ipynb
function get_unique_indices(x)
    
    #=
    This function returns the indices of unique elements in x.
    "i -> x[i], 1:length(x)" means we want to return the value i if and 
    only if x[i] is unique for i in 1 through length(x)
    
    TODO: 
    Extend to work as in the python code to return purged versions
    of border_terms_raw and border_evaluations_raw alongside the indices
    
    =#
    
    a = unique(i -> x[i], 1:length(x))
        
    return a
    
end

get_unique_indices (generic function with 1 method)

In [3]:
function mat_to_arr_of_arrs(A::Matrix, col_is_row::Int=0)
    #=
    This function takes as input a matrix-type object and returns it
    in form of an array of arrays as is. The rows of the matrix become 
    the rows of the array, i.e. the first row of A becomes the first array
    in the output.
    The optional parameter 'col_is_row' can be changed to 1 (default=0) if it is desired for
    matrix columns to become the individual arrays. 
    =#

    if  col_is_row == 0
        # converts matrix to array of arrays with rows of matrix being the individual entries of array,
        # i.e. first row of matrix A becomes the array result[1]
        result = [A[i,:] for i in 1:size(A,1)]
    
    
    elseif col_is_row == 1
        # converts matrix to array of arrays with columns of matrix being the individual entries of array,
        # i.e. first column of matrix A becomes the array result[1]
        result = C = [A[:,i] for i in 1:size(A,2)]


    else
        println("The optional argument 'col_is_row' was given an invalid value.")
        return 
        
    end
    
    return result
    
end

mat_to_arr_of_arrs (generic function with 2 methods)

In [4]:
function construct_border(
        terms::Vector, terms_eval::Vector, X_train::Vector, 
        degree_1_terms::Vector=[],  # These will eventually be SparseArrays
        degree_1_terms_eval::Vector=[], 
        purging_terms::Vector=[])
    
    #=
    This function constructs the border of terms. 
    Julia equivalent to same method in cgavi/src/feature_transformations/auxillary_functions_avi.py 
    
    NOT YET COMPLETE! Only basic functionality of constructing the border and finding duplicates. 
    The purging terms part still needs to be implemented. Also the parts that make sure all 
    inputs are of the same shape/dimension, i.e. the fd() function from the python code.
    =#
    
    if degree_1_terms == []
        # 'I' comes from the LinearAlgebra package
        border_terms_raw = mat_to_arr_of_arrs(
            1 * Matrix(I, length(X_train[1]), length(X_train[1]))) 
            
        border_terms_raw_evaluation = X_train
        
    else
        # This is a pretty ugly way to do this but I guess it works? Probably going to clean this up sometime.
        # Works in the same way as the cp.repeat() and cp.tile() part in the python code.
        terms_repeat = repeat(terms, inner=length(degree_1_terms[1]))
            
        l = length(terms_repeat)
        m = length(degree_1_terms)
        k = Int(l / m)
            
        degree_1_terms_tile = repeat(degree_1_terms, outer=k)
        
        
        border_terms_raw = degree_1_terms_tile + terms_repeat
        
        # Will insert the same thing for the evaluation part. Should not be too hard, just need to 
        # figure out how the dimensions will look
         
    end
    
    unique_indices = get_unique_indices(border_terms_raw)
    
    #= 
    TODO:
    Implement the purging of terms given an array of 'purging_terms'
    =#
    non_purging_indices = unique_indices
    
    return border_terms_raw, non_purging_indices
    
end

construct_border (generic function with 4 methods)

In [5]:
# Example
deg_1_terms = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
terms = [[3, 0, 0], [2, 1, 0], [2, 0, 1], [1, 2, 0], [0, 1, 2]]
border = construct_border(terms, [1], [], deg_1_terms)

([[4, 0, 0], [3, 1, 0], [3, 0, 1], [3, 1, 0], [2, 2, 0], [2, 1, 1], [3, 0, 1], [2, 1, 1], [2, 0, 2], [2, 2, 0], [1, 3, 0], [1, 2, 1], [1, 1, 2], [0, 2, 2], [0, 1, 3]], [1, 2, 3, 5, 6, 9, 11, 12, 13, 14, 15])

In [6]:
A = rand(1:10, 3, 3)

3×3 Matrix{Int64}:
 8  6   2
 3  9   3
 8  3  10

In [7]:
B = mat_to_arr_of_arrs(A)

3-element Vector{Vector{Int64}}:
 [8, 6, 2]
 [3, 9, 3]
 [8, 3, 10]